In [16]:
import pyodbc
import json
import pandas as pd
import numpy as np
import json

import warnings
warnings.filterwarnings("ignore")

from flask import Flask, request, jsonify
import os

In [5]:
import pandas as pd
print(pd.__version__)

2.0.3


In [12]:
from google.colab import drive
import json

# Mount your Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
class Make_Output:
    def __init__(self, sample_data_file_path, user_id):
        self.sample_data_file_path = sample_data_file_path
        self.user_id = user_id
        self.sample_data = pd.read_json(self.sample_data_file_path)

    def output_data(self):
        user_data = self.sample_data[self.sample_data['User ID'] == self.user_id]

        # Calculate 1M, 3M, and 1Y returns and volatility
        for ticker in user_data['Ticker'].unique():
            ticker_data = user_data[user_data['Ticker'] == ticker]
            ticker_data = ticker_data.sort_values('Trade_Date')

            # Calculate returns
            close_prices = ticker_data['Close_Price']
            returns_1m, returns_3m, returns_1y = self.calculate_returns(close_prices)

            # Calculate volatility (using daily returns for the last month)
            daily_returns = close_prices.pct_change()
            volatility_1m = self.calculate_volatility(daily_returns.tail(30))

            # Update the user_data DataFrame
            user_data.loc[user_data['Ticker'] == ticker, '1M Returns'] = returns_1m
            user_data.loc[user_data['Ticker'] == ticker, '3M Returns'] = returns_3m
            user_data.loc[user_data['Ticker'] == ticker, '1Y Returns'] = returns_1y
            user_data.loc[user_data['Ticker'] == ticker, '1M Volatility'] = volatility_1m

        return user_data

    def calculate_returns(self, price_series):
        # Assuming price_series is sorted in ascending order by date
        returns_1m = price_series.pct_change(periods=30)  # Assuming 30 days in a month
        returns_3m = price_series.pct_change(periods=90)  # Assuming 90 days in a quarter
        returns_1y = price_series.pct_change(periods=365) # Assuming 365 days in a year

        return returns_1m.iloc[-1], returns_3m.iloc[-1], returns_1y.iloc[-1]

    def calculate_volatility(self, daily_returns):
        volatility = daily_returns.std() * (252 ** 0.5)  # Assuming 252 trading days in a year
        return volatility

app = Flask(__name__)

@app.route('/get_user_data', methods=['GET'])
def get_user_data():
    user_id = request.args.get('user_id', default=None, type=str)
    if not user_id:
        return jsonify({"error": "User ID is required"}), 400

    file_path = '/content/drive/My Drive/sample_data.json'
    execute = Make_Output(file_path, user_id)
    output_df = execute.output_data()

    return jsonify(output_df.to_dict(orient='records'))

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
